In [115]:
import numpy as np
import pandas as pd
import warnings
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings('ignore')
import os
print(os.getcwd())
print(os.listdir("."))

d:\求职\Google实习\Week 5&Week 6\anime\Anime-Recommender-System-master
['.gitignore', 'anime.csv', 'animelist.csv', 'anime_with_synopsis.csv', 'Content Based Recommender.ipynb', 'image.png', 'Item Based Collaborative Filtering.ipynb', 'LICENSE', 'Rating Based Recommender.ipynb', 'rating_complete.csv', 'README.md', 'User Based Collaborative Filtering.ipynb', 'watching_status.csv']


#  Import Packages

# Read the data

In [116]:
anime = pd.read_csv("anime.csv")
user = pd.read_csv("animelist.csv")



In [117]:

print(anime.shape)
print(user.shape)


(17562, 35)
(109224747, 5)


In [118]:
synopsis = pd.read_csv("anime_with_synopsis.csv")
user.rename(columns = {'anime_id':'MAL_ID'}, inplace = True)
anime.rename(columns = {'Rating':'RRating'}, inplace = True)
anime.head()

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,...,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,...,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,...,50229.0,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,...,2182.0,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",Fall 2004,...,312.0,529.0,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0


In [119]:
synopsis.head()

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...


In [120]:
synopsis.shape

(16214, 5)

In [121]:
user.head()

,user_id,MAL_ID,rating,watching_status,watched_episodes
0,0,67,9,1,1
1,0,6702,7,1,4
2,0,242,10,1,4
3,0,4898,0,1,1
4,0,21,10,1,0


In [122]:
user.shape

(109224747, 5)

# Replacing missing rating with NaN

In [123]:
anime= anime.replace('Unknown', np.nan)
#only select the anime with the genre of TV and Movie
#only select the anime with over 75% percentile populatity (which means we only recommend famous movie)

anime = anime[(anime['Type'] == 'TV') | (anime['Type'] == 'Movie')]
m = anime['Members'].quantile(0.75)
anime = anime[(anime['Members'] >= m)]
user= user.replace('Unknown', np.nan)
synopsis=synopsis.replace('Unknown', np.nan)
user = user[user['user_id'] <= 10000]
user.shape

(2966491, 5)

In [149]:
anime

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,...,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",NaN,...,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,...,50229.0,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,...,2182.0,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0
5,15,Eyeshield 21,7.95,"Action, Sports, Comedy, Shounen",NaN,アイシールド21,TV,145,"Apr 6, 2005 to Mar 19, 2008",Spring 2005,...,9226.0,14904.0,22811.0,16734.0,6206.0,2621.0,795.0,336.0,140.0,151.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17191,42963,"Kanojo, Okarishimasu 2nd Season",NaN,"Comedy, Romance, School, Shounen",NaN,彼女、お借りします,TV,NaN,NaN,NaN,...,5.0,2.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
17224,43299,Wonder Egg Priority,8.32,"Psychological, Drama, Fantasy",NaN,ワンダーエッグ・プライオリティ,TV,12,"Jan 13, 2021 to ?",Winter 2021,...,7564.0,12054.0,12657.0,5626.0,1516.0,682.0,258.0,113.0,77.0,144.0
17272,43608,Kaguya-sama wa Kokurasetai: Tensai-tachi no Re...,NaN,"Comedy, Psychological, Romance, School, Seinen",NaN,かぐや様は告らせたい ～天才たちの恋愛頭脳戦～,TV,NaN,NaN,NaN,...,3.0,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
17519,47778,Kimetsu no Yaiba: Yuukaku-hen,NaN,"Action, Historical, Demons, Supernatural, Shounen",NaN,鬼滅の刃 遊郭編,TV,NaN,2021 to ?,NaN,...,11.0,2.0,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [125]:
print(anime.shape)

(2010, 35)


# Create index for anime name

In [166]:
anime_index = pd.Series(anime.index, index=anime.Name)
anime_index

Name
Cowboy Bebop                                                                 0
Cowboy Bebop: Tengoku no Tobira                                              1
Trigun                                                                       2
Witch Hunter Robin                                                           3
Eyeshield 21                                                                 5
                                                                         ...  
Kanojo, Okarishimasu 2nd Season                                          17191
Wonder Egg Priority                                                      17224
Kaguya-sama wa Kokurasetai: Tensai-tachi no Renai Zunousen 3rd Season    17272
Kimetsu no Yaiba: Yuukaku-hen                                            17519
Hataraku Maou-sama! 2nd Season                                           17540
Length: 2010, dtype: int64

# Join the data

In [150]:
joined=pd.merge(pd.merge(anime,user,how='inner',on='MAL_ID'),synopsis,how="inner",on="MAL_ID")


In [151]:
joined.shape

(2154039, 43)

# Create a pivot table

In [152]:
joined1 = joined[['user_id', 'Name_x', 'rating']]

In [153]:
joined1.head()

,user_id,Name_x,rating
0,3,Cowboy Bebop,9
1,6,Cowboy Bebop,6
2,8,Cowboy Bebop,0
3,13,Cowboy Bebop,0
4,14,Cowboy Bebop,9


In [154]:
pivot = pd.pivot_table(joined1, index='Name_x', columns='user_id', values='rating')
pivot.head()

user_id,0,1,2,3,4,5,6,7,8,9,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
Name_x,,,,,,,,,,,,,,,,,,,,,
"""Bungaku Shoujo"" Movie",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//Roots,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//Sign,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//Tasogare no Udewa Densetsu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
07-Ghost,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,7.0,NaN,9.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [155]:
pivot.shape

(2010, 9141)

# Drop all users that never rate an anime

In [156]:
pivot.dropna(axis=1, how='all', inplace=True)
pivot.head()

user_id,0,1,2,3,4,5,6,7,8,9,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
Name_x,,,,,,,,,,,,,,,,,,,,,
"""Bungaku Shoujo"" Movie",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//Roots,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//Sign,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//Tasogare no Udewa Densetsu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
07-Ghost,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,7.0,NaN,9.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [157]:
pivot.shape

(2010, 9141)

# Center the mean around 0 (centered cosine / pearson)

In [158]:
pivot_norm = pivot.apply(lambda x: x - np.nanmean(x), axis=1)
pivot_norm.shape

(2010, 9141)

# Item Based Collaborative Filtering

In [159]:
# fill NaN with 0
pivot_norm.fillna(0, inplace=True)
pivot_norm.head()

user_id,0,1,2,3,4,5,6,7,8,9,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
Name_x,,,,,,,,,,,,,,,,,,,,,
"""Bungaku Shoujo"" Movie",0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
.hack//Roots,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
.hack//Sign,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,-3.695421,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
.hack//Tasogare no Udewa Densetsu,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
07-Ghost,0.0,0.0,0.0,0.0,-4.283951,0.0,0.0,0.0,0.0,0.0,...,2.716049,0.0,4.716049,0.0,0.0,0.0,0.0,0.0,-4.283951,0.0


## Calculate Similar Items

In [160]:
# convert into dataframe to make it easier
item_sim_df = pd.DataFrame(cosine_similarity(pivot_norm, pivot_norm), index=pivot_norm.index, columns=pivot_norm.index)
item_sim_df.head()

Name_x,"""Bungaku Shoujo"" Movie",.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,07-Ghost,100-man no Inochi no Ue ni Ore wa Tatteiru,11eyes,18if,2.43: Seiin Koukou Danshi Volley-bu,3-gatsu no Lion,...,Zombie-Loan,Zombieland Saga,Zombieland Saga: Revenge,Zutto Mae kara Suki deshita.: Kokuhaku Jikkou Iinkai,ef: A Tale of Melodies.,ef: A Tale of Memories.,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei,xxxHOLiC Movie: Manatsu no Yoru no Yume
Name_x,,,,,,,,,,,,,,,,,,,,,
"""Bungaku Shoujo"" Movie",1.000000,0.029049,0.057868,0.033295,0.080655,0.034405,0.121553,0.025593,-0.008593,0.057976,...,0.088702,0.066622,0.0,0.082452,0.166179,0.183204,0.039849,0.087812,0.072903,0.084739
.hack//Roots,0.029049,1.000000,0.395578,0.449194,0.132589,0.019658,0.106209,0.042527,0.004456,0.029139,...,0.095467,0.014495,0.0,0.024283,0.038435,0.039517,0.091855,0.110149,0.093594,0.089520
.hack//Sign,0.057868,0.395578,1.000000,0.331438,0.119513,0.012310,0.110675,0.035146,0.001136,0.020838,...,0.088614,0.042351,0.0,0.027587,0.084438,0.085091,0.157815,0.110079,0.069045,0.057047
.hack//Tasogare no Udewa Densetsu,0.033295,0.449194,0.331438,1.000000,0.116383,0.009544,0.105932,0.044054,-0.000876,0.018106,...,0.103517,0.029626,0.0,0.025646,0.059739,0.060018,0.131032,0.074616,0.086401,0.071850
07-Ghost,0.080655,0.132589,0.119513,0.116383,1.000000,0.041472,0.182050,0.038297,0.012390,0.049228,...,0.151213,0.038003,0.0,0.054123,0.085327,0.075466,0.093458,0.124979,0.083104,0.078117


In [161]:
def get_similar_anime(anime_name):
    if anime_name not in pivot_norm.index:
        return None, None
    else:
        sim_animes = item_sim_df.sort_values(by=anime_name, ascending=False).index[1:]
        sim_score = item_sim_df.sort_values(by=anime_name, ascending=False).loc[:, anime_name].tolist()[1:]
        return sim_animes, sim_score

In [162]:
animes, score = get_similar_anime("Cowboy Bebop")
for x,y in zip(animes[:10], score[:10]):
    print("{} with similarity of {}".format(x, y))

Samurai Champloo with similarity of 0.3123013556681629
Neon Genesis Evangelion with similarity of 0.2756220399765736
Cowboy Bebop: Tengoku no Tobira with similarity of 0.2596214755110671
Koukaku Kidoutai with similarity of 0.25549016361830046
Trigun with similarity of 0.2554242300604982
Black Lagoon with similarity of 0.24851248032050238
Akira with similarity of 0.23403281792440714
Tengen Toppa Gurren Lagann with similarity of 0.2315774928129124
Baccano! with similarity of 0.22456625266372018
Koukaku Kidoutai: Stand Alone Complex with similarity of 0.20307103409426075


## Helper Function

In [163]:
# predict the rating of anime x by user y
def predict_rating(user_id, anime_name, max_neighbor=10):
    animes, scores = get_similar_anime(anime_name)
    anime_arr = np.array([x for x in animes])
    sim_arr = np.array([x for x in scores])
    
    # select only the anime that has already rated by user x
    filtering = pivot_norm[user_id].loc[anime_arr] != 0
    
    # calculate the predicted score
    s = np.dot(sim_arr[filtering][:max_neighbor], pivot[user_id].loc[anime_arr[filtering][:max_neighbor]]) \
            / np.sum(sim_arr[filtering][:max_neighbor])
    
    return s

In [164]:
predict_rating(3, "Steins;Gate")

7.280991222627498

In [165]:
predict_rating(3, "Cowboy Bebop")

7.061242485166327

## Get Recommendation

In [177]:
# recommend top n_anime for user x based on item collaborative filtering algorithm
def get_recommendation(user_id, n_anime=10):
    predicted_rating = np.array([])
    
    for _anime in pivot_norm.index:
        predicted_rating = np.append(predicted_rating, predict_rating(user_id, _anime))
    
    # don't recommend something that user has already rated
    temp = pd.DataFrame({'predicted':predicted_rating, 'name':pivot_norm.index})
    filtering = (pivot_norm[user_id] == 0.0)
    temp = temp.loc[filtering.values].sort_values(by='predicted', ascending=False)

    # recommend n_anime anime
    return pd.merge(anime.loc[anime_index.loc[temp.name[:n_anime]]],synopsis[['MAL_ID',"sypnopsis"]],how="inner",on="MAL_ID")[['MAL_ID','Name','Score','Type','Source','sypnopsis']]
    

In [178]:
anime

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,...,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",NaN,...,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,...,50229.0,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,...,2182.0,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0
5,15,Eyeshield 21,7.95,"Action, Sports, Comedy, Shounen",NaN,アイシールド21,TV,145,"Apr 6, 2005 to Mar 19, 2008",Spring 2005,...,9226.0,14904.0,22811.0,16734.0,6206.0,2621.0,795.0,336.0,140.0,151.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17191,42963,"Kanojo, Okarishimasu 2nd Season",NaN,"Comedy, Romance, School, Shounen",NaN,彼女、お借りします,TV,NaN,NaN,NaN,...,5.0,2.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
17224,43299,Wonder Egg Priority,8.32,"Psychological, Drama, Fantasy",NaN,ワンダーエッグ・プライオリティ,TV,12,"Jan 13, 2021 to ?",Winter 2021,...,7564.0,12054.0,12657.0,5626.0,1516.0,682.0,258.0,113.0,77.0,144.0
17272,43608,Kaguya-sama wa Kokurasetai: Tensai-tachi no Re...,NaN,"Comedy, Psychological, Romance, School, Seinen",NaN,かぐや様は告らせたい ～天才たちの恋愛頭脳戦～,TV,NaN,NaN,NaN,...,3.0,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
17519,47778,Kimetsu no Yaiba: Yuukaku-hen,NaN,"Action, Historical, Demons, Supernatural, Shounen",NaN,鬼滅の刃 遊郭編,TV,NaN,2021 to ?,NaN,...,11.0,2.0,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [179]:

get_recommendation(3)

,MAL_ID,Name,Score,Type,Source,sypnopsis
0,30484,Steins;Gate 0,8.51,TV,Visual novel,"The eccentric, self-proclaimed mad scientist R..."
1,33049,Fate/stay night Movie: Heaven's Feel - II. Los...,8.59,Movie,Visual novel,"The Fifth Holy Grail War continues, and the en..."
2,39491,Psycho-Pass 3,7.54,TV,Original,"Thanks to the Sibyl System, the mental states ..."
3,25537,Fate/stay night Movie: Heaven's Feel - I. Pres...,8.26,Movie,Visual novel,The Holy Grail War: a violent battle between m...
4,35180,3-gatsu no Lion 2nd Season,9.0,TV,Manga,"Now in his second year of high school, Rei Kir..."
5,164,Mononoke Hime,8.72,Movie,Original,hen an Emishi village is attacked by a fierce ...
6,35839,Sora yori mo Tooi Basho,8.57,TV,Original,Filled with an overwhelming sense of wonder fo...
7,37441,Psycho-Pass: Sinners of the System Case.2 - Fi...,7.48,Movie,Original,"Enforcer Teppei Sugou, former ace pilot of the..."
8,41433,Akudama Drive,7.75,TV,Original,"The bustling metropolis of Kansai, where cyber..."
9,35677,Liz to Aoi Tori,8.22,Movie,Novel,z's days of solitude come to an end when she m...


In [180]:
get_recommendation(5)

,MAL_ID,Name,Score,Type,Source,sypnopsis
0,431,Howl no Ugoku Shiro,8.67,Movie,Novel,"That jumbled piece of architecture, that cacop..."
1,2472,Naruto: Shippuuden Movie 1,7.29,Movie,Manga,group of ninja is planning to revive a powerfu...
2,523,Tonari no Totoro,8.31,Movie,Original,"In 1950s Japan, Tatsuo Kusakabe relocates hims..."
3,5460,Detective Conan Movie 13: The Raven Chaser,8.24,Movie,Manga,Kudou Shinichi is living his life as Edogawa C...
4,164,Mononoke Hime,8.72,Movie,Original,hen an Emishi village is attacked by a fierce ...
5,2142,Blue Dragon,6.41,TV,Game,s Shu's village was being attacked by an unkno...
6,1361,Final Fantasy: The Spirits Within,6.37,Movie,Game,"The year is 2065, and the planet of Earth is f..."
7,416,Kurenai no Buta,7.98,Movie,Manga,"fter a curse turned him into a pig, World War ..."
8,417,Ragnarök The Animation,6.47,TV,Game,great evil is sweeping over the realm; an evil...
9,7711,Karigurashi no Arrietty,7.94,Movie,Novel,"hile spending the summer at his aunt's house, ..."
